# КП 2. Лінійний криптоаналіз блокових шифрів.

Виконали студенти групи ФІ-32мн Костянтин Баєвський та Денис Шифрін. Варіант обрали 16, за номером Шифріна в списку групи.

## Мета роботи

Опанування сучасних методів криптоаналізу блокових шифрів, набуття навичок у дослідженні стійкості блокових шифрів до лінійного криптоаналізу.

## Опис труднощів, що виникали, та шляхів їх розв’язання

1. Алгоритм пошуку високоімовірних диференціалів, M2 та згодом пошук ключів також не були тривіальним, тому знадобився час на коректне написання.
2. Ми звикли писати мовами програмування із строгою типізацією, Java та C#, але виникло багато проблем в роботі із файлами та бітами, тому було прийнято рішення перейти на python для виконання даного практикуму.
3. Нам досі не до кінця зрозумілі принципи роботи з потоками та процесами в python, але після купи гайдів ми таки змогли здобути достатньо знань для написання практикуму.
4. Взаємодія із представленим застосунком "heys.exe". Потрібно було викликати програму мовою програмування, а потім симулювати натискання клавіші ентер, щоб закрити пул. 
5. Час очікування результатів роботи на наших обчислювальних машинах був досить великим, через слабке залізо.

## Етап підготовки

### Необхідні імпорти

In [1]:
import subprocess
from collections import Counter
from itertools import product
from math import prod
from random import randrange
import random
import numpy as np
import pandas as pd
from multiprocess import Pool

## Завдання 1. 

Реалізувати шестираундовий шифр Хейса. Ключем шифрування вважати довільний 112-бітний бітовий рядок, який розбивається на сім незалежних 16-бітних раундових підключа.

In [2]:
class Heys:
    def __init__(self):
        self.s_block = [0x4, 0x3, 0xE, 0xD, 0x5, 0x0, 0x2, 0xB, 0x1, 0xA, 0x7, 0x6, 0x9, 0xF, 0x8, 0xC]
        self.s_block_inverse = [0x5, 0x8, 0x6, 0x1, 0x0, 0x4, 0xB, 0xA, 0xE, 0xC, 0x9, 0x7, 0xF, 0x3, 0x2, 0xD]

    def round(self, block: int, round_key: int=0x0):
        y = block ^ round_key
        y_s = self.substitute(y)
        y_p = self.permute(y_s)
        return y_p

    def substitute(self, block: int):
        return (self.s_block[(block >> 12) & 0xF] << 12) | (self.s_block[(block >> 8) & 0xF] << 8) |\
             (self.s_block[(block >> 4) & 0xF] << 4) | (self.s_block[block & 0xF])
    
    def substitute_inverse(self, block: int):
        return (self.s_block_inverse[(block >> 12) & 0xF] << 12) | (self.s_block_inverse[(block >> 8) & 0xF] << 8) |\
             (self.s_block_inverse[(block >> 4) & 0xF] << 4) | (self.s_block_inverse[block & 0xF])

    def permute(self, block: int):
        result = 0
        for i in range(4):
            for j in range(4):
                result |= (block >> (i * 4 + j) & 1) << (j * 4 + i) 
        return result


## Завдання 2. 

Реалізувати пошук високоімовірних п’ятираундових диференціалів шифру Хейса методом "гілок та границь".

### Таблиця диференціальних імовірностей

In [3]:
heys = Heys()

def get_table():
    heys = Heys()
    linear_potential = np.zeros((16, 16))

    for alpha in range(16):
        for beta in range(16):
            for x in range(16):
                exponent = (bin(alpha & x).count('1') % 2) ^ (bin(beta & heys.substitute(x)).count('1') % 2)
                linear_potential[alpha][beta] += (-1) ** exponent


    #pd.DataFrame(linear_potential)
    linear_potential /= 16
    linear_potential **= 2
    
    return linear_potential
linear_potential = get_table()
pd.DataFrame(linear_potential)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,0.0,0.0000,0.0625,0.0625,0.0000,0.0000,0.0625,0.0625,0.0625,0.0625,0.0000,0.0000,0.0625,0.0625,0.2500,0.2500
2,0.0,0.0625,0.0625,0.0000,0.0625,0.0000,0.0000,0.0625,0.0625,0.0000,0.2500,0.0625,0.0000,0.0625,0.0625,0.2500
3,0.0,0.0625,0.2500,0.0625,0.0625,0.2500,0.0625,0.0000,0.0000,0.0625,0.0000,0.0625,0.0625,0.0000,0.0625,0.0000
4,0.0,0.0000,0.0625,0.0625,0.0625,0.0625,0.0000,0.0000,0.0625,0.0625,0.0000,0.2500,0.0000,0.2500,0.0625,0.0625
5,0.0,0.0000,0.0000,0.2500,0.0625,0.0625,0.0625,0.0625,0.0000,0.0000,0.2500,0.0000,0.0625,0.0625,0.0625,0.0625
6,0.0,0.0625,0.0000,0.0625,0.2500,0.0625,0.2500,0.0625,0.0000,0.0625,0.0000,0.0625,0.0000,0.0625,0.0000,0.0625
7,0.0,0.0625,0.0625,0.0000,0.0000,0.0625,0.0625,0.0000,0.0625,0.2500,0.0000,0.0625,0.0625,0.2500,0.0000,0.0625
8,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.2500,0.2500,0.0625,0.0625,0.0625,0.0625,0.0625,0.0625,0.0625,0.0625
9,0.0,0.2500,0.0625,0.0625,0.2500,0.0000,0.0625,0.0625,0.0000,0.0000,0.0625,0.0625,0.0000,0.0000,0.0625,0.0625


### 2.1. Алгоритм пошуку високоімовірних диференціалів ітеративних шифрів

$$
\begin{array}{l}
1. \ \Gamma_0 (\alpha) := \{ (\alpha, 1) \}. \\
2. \ \text{Для всіх } t \text{ від } 1 \text{ до } r \text{ виконати:} \\
    \quad 2.1 \ \Gamma_t (\alpha) := \varnothing. \\
    \quad 2.2 \ \text{Для кожної пари } (\beta_i, p_i) \in \Gamma_{t-1} (\alpha) \text{ виконати:} \\
    \quad \quad \text{// побудова «гілок»} \\
    \quad \quad 2.2.1 \ \text{Для кожної пари } (\gamma_j, q_j) \in \Delta (\beta_i) \text{ виконати:} \\
        \quad \quad \quad 2.2.1.1 \ \text{Якщо } (\gamma_j, p(\gamma_j)) \in \Gamma_t (\alpha), \text{ то:} \\
            \quad \quad \quad \quad \text{а.} \ p(\gamma_j) = p(\gamma_j) + p_i \cdot q_j; \\
        \quad \quad \quad 2.2.1.2 \ \text{інакше:} \\
            \quad \quad \quad \quad \text{а.} \ \text{включити у } \Gamma_t (\alpha) \text{ пару } (\gamma_j, p_i \cdot q_j). \\
    \quad 2.3 \ \text{Для кожної пари } (\gamma_i, p_i) \in \Gamma_t (\alpha) \text{ виконати:} \\
    \quad \quad \text{// перевірка «границь»} \\
        \quad \quad 2.3.1 \ \text{Якщо } p_i \leq p^*, \text{ то:} \\
            \quad \quad \quad \text{а.} \ \text{вилучити пару } (\gamma_i, p_i) \text{ з } \Gamma_t (\alpha). \\
\end{array}
$$



Сенс алгоритму полягає в тому, що основна задача гілкується, тобто розбивається на менші підзадачі, та обмежується границями. В залежності від значень границь задача може бути відкинута, чи розбита знов. Процес відбувається доти, доки існують менші підзадачі.

Для оптимізації також можна використати передобчислені диференціали в минулому пункті.

In [6]:
local_diff_table = {}

def get_differences(alpha):
    alpha_list = [(alpha >> (4 * i)) & 15 for i in range(4)]
    non_null_betas = [[beta for beta in range(16) if linear_potential[alpha][beta] != 0] for alpha in alpha_list]

    beta_counts = Counter()
    for beta_list in product(*non_null_betas):
        beta = sum([beta_list[i] << (i * 4) for i in range(4)])
        probability = prod([linear_potential[alpha_list[i]][beta_list[i]] for i in range(4)])
        beta_counts[beta] += probability

    return beta_counts

def diffSearch(alpha):
    global local_diff_table

    # Пункт 1.
    g_prev = {alpha: 1}
    
    # Пункт 2.
    for _ in range(1, 6):
        # Пункт 2.1.
        g_current = {}

        # Пункт 2.2.
        for beta, p_i in g_prev.items():
            if beta not in local_diff_table:
                local_diff_table[beta] = get_differences(beta)

            # Пункт 2.2.1
            for gamma, q_j in local_diff_table[beta].items():
                tmp_prob = p_i * q_j
                if gamma in g_current:
                    # Пункт 2.2.1.1
                    g_current[gamma] += tmp_prob
                else:
                    # Пункт 2.2.1.2
                    g_current[gamma] = tmp_prob

        # Пункт 2.3
        g_new_filtered = {}
        for gamma, p_i in g_current.items():
            # Пункт 2.3.1
            if p_i > 0.00015: # встановлення границі p^{*} = 0.00015
                g_new_filtered[heys.permute(gamma)] = p_i

        g_prev = g_new_filtered

    return (alpha, g_prev)

In [7]:
%%time
local_diff_table = {}

def prepare_alphas():
    alfas = []
    for i in range(4):
        for j in range(1, 16):
            alfas.append(j << (4 * i))
    return alfas


candidates = [diffSearch(alpha) for alpha in prepare_alphas()]
    

print('Possible candidates after "branches and bounds" search, candidates len: {0}\n candidates: \n'.format(
    len(candidates)))
list(candidates)

Possible candidates after "branches and bounds" search, candidates len: 60
 candidates: 

CPU times: total: 6min 26s
Wall time: 6min 31s


[(1, {}),
 (2, {}),
 (3, {}),
 (4, {}),
 (5, {}),
 (6, {}),
 (7, {}),
 (8, {}),
 (9, {}),
 (10, {}),
 (11, {}),
 (12, {}),
 (13, {}),
 (14, {}),
 (15, {}),
 (16, {34952: 0.00015535950660705566}),
 (32, {2184: 0.00019888300448656082, 2176: 0.00017118453979492188}),
 (48,
  {8738: 0.0001857355237007141,
   2184: 0.0002312178839929402,
   32768: 0.0002504747826606035,
   32896: 0.0002113695372827351,
   32904: 0.00022464990615844727,
   34824: 0.00025004148483276367,
   34944: 0.0002116858959197998,
   34952: 0.00035111262695863843,
   2176: 0.00016558170318603516,
   136: 0.00017853197641670704}),
 (64, {34952: 0.0001729428768157959}),
 (80,
  {2184: 0.00016917288303375244,
   32896: 0.00016304850578308105,
   34952: 0.00017976760864257812}),
 (96,
  {32904: 0.0002490729093551636,
   34824: 0.0002490729093551636,
   34952: 0.0001951158046722412}),
 (112,
  {32904: 0.000173165462911129,
   34824: 0.00020357873290777206,
   34944: 0.00018776953220367432,
   34952: 0.0002358546480536461,
  

In [22]:
possible_candidates = []
for alpha, betas in list(candidates):
    for beta, prob in betas.items():
        possible_candidates.append(((alpha, beta), prob))
possible_candidates = sorted(possible_candidates, key=lambda x: x[1], reverse=True)
possible_candidates[:30]

[((224, 34952), 0.0004047989204991609),
 ((48, 34952), 0.00035111262695863843),
 ((160, 34952), 0.0003030932566616684),
 ((144, 34952), 0.0002830028533935547),
 ((160, 2176), 0.00028231891337782145),
 ((224, 34824), 0.00027692317962646484),
 ((224, 34944), 0.0002740621566772461),
 ((208, 2176), 0.00027164630591869354),
 ((208, 2184), 0.000268660020083189),
 ((160, 2184), 0.00026464826078154147),
 ((224, 2184), 0.00026377433096058667),
 ((224, 32768), 0.0002634284319356084),
 ((144, 32896), 0.00025197863578796387),
 ((48, 32768), 0.0002504747826606035),
 ((48, 34824), 0.00025004148483276367),
 ((192, 2176), 0.00025000167079269886),
 ((96, 32904), 0.0002490729093551636),
 ((96, 34824), 0.0002490729093551636),
 ((144, 2184), 0.0002477318048477173),
 ((224, 32904), 0.00023734569549560547),
 ((112, 34952), 0.0002358546480536461),
 ((48, 2184), 0.0002312178839929402),
 ((48, 32904), 0.00022464990615844727),
 ((144, 32904), 0.0002240389585494995),
 ((144, 34824), 0.0002240389585494995),
 ((19

## Завдання 3. 

Реалізувати атаку на перший раундовий ключ шифру Хейса за такою схемою.

Для реалізації атаки необхідно:
- а. Одержати необхідну кількість пар «відкритий текст-шифротекст».
- б. Для кожної апроксимації реалізувати алгоритм атаки М2. Відмітити кожний кандидат у ключі, для якого лічильник алгоритму М2 перевищив певний поріг (значення
цього порогу залишається на ваш розсуд; наприклад, ви можете розглядати першу десятку, перші 50 або перші 100 ключів).
- в. Серед усіх відмічених кандидатів для усіх знайдених апроксимацій обрати ті, які найчастіше обирались алгоритмом М2.

### а. Одержати необхідну кількість пар «відкритий текст-шифротекст».

In [23]:
def read_file(file):
    with open(file, 'rb') as f:
        data = f.read()
    return [(data[i+1] << 8) | data[i] for i in range(0, len(data), 2)]

In [24]:
def write_file(file, text: list[int]):
    with open(file, 'wb') as f:
        f.write(bytearray([byte for x in text for byte in x.to_bytes(2, byteorder='little')]))

In [25]:
def get_ciphertext(filefrom, fileto):
    subprocess.Popen(fr'.\heys.exe e 16 {filefrom} {fileto}', shell=True, stdout=subprocess.DEVNULL, stderr=None).communicate(input="\x0a".encode())

    return read_file(fileto)

In [26]:
def get_unique_alpha_beta_values(candidates):
    return set([alpha for (alpha, _), _ in candidates]).union(set([beta for (_, beta), _ in candidates]))

In [27]:
def generate_ciphertext(samples_num, no):
    random.seed(0)
    plaintext = [randrange(0, 2**16) for _ in range(0, samples_num)]

    write_file(fr'./files/plain_{no}.bin', plaintext) 
    write_file(fr'./files/cipher_{no}.bin', "")

    ciphertext = get_ciphertext(fr'./files/plain_{no}.bin', fr'./files/cipher_{no}.bin')

    with Pool() as pool:
        heys_round = list(
            pool.map(lambda k: [heys.permute(heys.substitute(x ^ k)) for x in plaintext], keys))

    return heys_round, ciphertext

### б. Для кожної апроксимації реалізувати алгоритм атаки М2.

In [28]:
def check_candidate(heys_round_texts, ct, alpha, beta):
    # Пункт 2.2
    u_k = 0
    for (ct1, ct2) in zip(heys_round_texts, ct):
        u_k += precomputed_custom_mul[alpha][ct1] ^ precomputed_custom_mul[beta][ct2]
    return abs(7000 - 2 * u_k)

#### Перевірка ключів.

In [29]:
%%time

keys = [i for i in range(0, 2**16)]

# precompute values for computing u(k) value
precomputed_custom_mul = dict(
    [(x, [(bin(x & k).count('1') % 2) for k in keys]) for x in get_unique_alpha_beta_values(possible_candidates)])



CPU times: user 289 ms, sys: 0 ns, total: 289 ms
Wall time: 289 ms


#### М2.

$$
\textbf{Алгоритм M2.} \\
\text{0. Нехай для $r$-раундового шифру $E$ відома лінійна апроксимація його останніх} \\
\quad \text{$r-1$ раундів } \alpha \cdot X_1 \oplus \beta \cdot X_r = 0 \text{ із суттєвою кореляцією } \varepsilon. \\
\quad \text{Алгоритм дозволяє знайти істинне значення ключа першого раунду шифрування } k_1. \\
\text{1. Одержати $N$ пар відкритих та зашифрованих текстів $(X, Y)$, де } Y = E_K(X). \\
\text{2. Для кожного кандидата $k$ у ключі $k_1$ виконати такі дії:} \\
\quad \text{2.1. Зашифрувати відкриті тексти $X$ на один раунд: } X_1 = F_1(X, k). \\
\quad \text{2.2. Обчислити значення} \\
\quad \quad \hat{u}(k) = \left| \left\{ (X_1, Y) : \alpha \cdot X_1 \oplus \beta \cdot Y = 0 \right\} \right| - \left| \left\{ (X_1, Y) : \alpha \cdot X_1 \oplus \beta \cdot Y = 1 \right\} \right|. \\
\text{3. Ключ } k_1 \text{ визначається як } k_1 = \arg\max_k \hat{u}(k).
$$

In [30]:
%%time

# Пункт 1 
heys_round, ciphertext = generate_ciphertext(7000, 1)
possible_key_candidates = {}

for (alpha, beta), _ in possible_candidates:
    
    # Пункт 2
    with Pool() as pool:
        # Пункт 2.1 та 2.2
        success_rates = list(pool.map(lambda k: check_candidate(heys_round[k], ciphertext, alpha, beta), keys))

        # Встановлюємо кількість кандидатів, яких ми хочемо відібрати
        candidates_amount = 100
        possible_max_candidates = []

        # Проходимо по можливих значеннях успішності у зворотному порядку (від максимального до мінімального)
        for max_u_k_value in range(max(success_rates), 0, -1):
            # Знаходимо всі ключі з поточною максимальною успішністю
            keys_with_max_u_k = [k for k, u_k in enumerate(success_rates) if u_k == max_u_k_value]

            # Якщо кількість таких ключів менша або дорівнює залишку кандидатів, додаємо їх всіх
            if len(keys_with_max_u_k) <= candidates_amount:
                possible_max_candidates += keys_with_max_u_k
                candidates_amount -= len(keys_with_max_u_k)
            else:
                # Інакше випадково обираємо потрібну кількість ключів та завершуємо відбір
                possible_max_candidates += random.sample(keys_with_max_u_k, candidates_amount)
                break

        possible_key_candidates[(alpha, beta)] = possible_max_candidates

# Пункт 3
best_candidate = max(possible_key_candidates, key=lambda k: max(possible_key_candidates[k]))


CPU times: user 55.5 s, sys: 5min 22s, total: 6min 17s
Wall time: 18min 49s


### в. Серед усіх відмічених кандидатів для усіх знайдених апроксимацій обрати ті, які найчастіше обирались алгоритмом М2.

Отримали наступний список із потенційними ключами серед яких є правильний ключ.

In [31]:
sorted_key_candidates = {}

for (_, _), keys in possible_key_candidates.items():
    for key in keys:
        if key in sorted_key_candidates:
            sorted_key_candidates[key] += 1
        else:
            sorted_key_candidates[key] = 1

sorted_key_candidates = sorted(sorted_key_candidates.items(), key=lambda x: x[1], reverse=True)
[(hex(key), count) for key, count in sorted_key_candidates[:100]]

[('0x448c', 3),
 ('0x6e8f', 3),
 ('0x65af', 3),
 ('0x671f', 3),
 ('0x611f', 3),
 ('0x62df', 3),
 ('0x5664', 3),
 ('0x6187', 3),
 ('0x150e', 3),
 ('0x1c05', 3),
 ('0x1c0e', 3),
 ('0xd96f', 3),
 ('0x3a2', 2),
 ('0x3af', 2),
 ('0x5dd4', 2),
 ('0x5dd8', 2),
 ('0x5ddf', 2),
 ('0x9cd5', 2),
 ('0x9cde', 2),
 ('0x3b71', 2),
 ('0x56d4', 2),
 ('0xbc6', 2),
 ('0xecc6', 2),
 ('0x91c6', 2),
 ('0xa2c6', 2),
 ('0x7ac6', 2),
 ('0xb1c6', 2),
 ('0x4a86', 2),
 ('0x4485', 2),
 ('0x8a4', 2),
 ('0x5a0', 2),
 ('0x609f', 2),
 ('0x6d7f', 2),
 ('0x6c2f', 2),
 ('0x6e2f', 2),
 ('0x64df', 2),
 ('0x642f', 2),
 ('0x6f2f', 2),
 ('0x684f', 2),
 ('0x624f', 2),
 ('0x6cdf', 2),
 ('0x63af', 2),
 ('0x6c9f', 2),
 ('0x691f', 2),
 ('0x6f3f', 2),
 ('0x67bf', 2),
 ('0x61df', 2),
 ('0x67ff', 2),
 ('0x664f', 2),
 ('0x60bf', 2),
 ('0x612f', 2),
 ('0x676f', 2),
 ('0x635f', 2),
 ('0x6a4f', 2),
 ('0x653f', 2),
 ('0x605f', 2),
 ('0x6f6f', 2),
 ('0x681f', 2),
 ('0x6c7f', 2),
 ('0x621f', 2),
 ('0x626f', 2),
 ('0x600f', 2),
 ('0x68bf', 2

In [32]:
possible_key_candidates

{(224, 34952): [38048,
  38049,
  38050,
  38051,
  38052,
  38053,
  38054,
  38055,
  38056,
  38057,
  38058,
  38059,
  38060,
  38061,
  38062,
  38063,
  928,
  929,
  930,
  931,
  932,
  933,
  934,
  935,
  936,
  937,
  938,
  939,
  940,
  941,
  942,
  943,
  24016,
  24017,
  24018,
  24019,
  24020,
  24021,
  24022,
  24023,
  24024,
  24025,
  24026,
  24027,
  24028,
  24029,
  24030,
  24031,
  40144,
  40145,
  40146,
  40147,
  40148,
  40149,
  40150,
  40151,
  40152,
  40153,
  40154,
  40155,
  40156,
  40157,
  40158,
  40159,
  22231,
  22234,
  22239,
  9382,
  9376,
  15231,
  22232,
  9379,
  9377,
  9385,
  22233,
  15230,
  9384,
  9388,
  15227,
  22230,
  9378,
  22236,
  9381,
  15217,
  15222,
  22235,
  22228,
  22238,
  9380,
  15228,
  15226,
  15225,
  22226,
  22225,
  9391,
  15221,
  9390,
  22227,
  15220,
  15223],
 (48, 34952): [3014,
  50374,
  56262,
  28870,
  46278,
  47814,
  57542,
  11462,
  31942,
  56518,
  12998,
  4806,
  12742,
 

## Висновки

В результаті виконання практикуму наведена реалізація атаки на перший раундовий ключ шифру Хейса. На практиці було перевірено стійкість данного шифру до статистичних атак.